In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
# soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [3]:
"""
url = f'https://www.yesstyle.com/es/mujer-limpiadores-faciales/list.html/bcc.14335_bpt.46#/bpt=46&sb=136&pn=1&l=17&s=10&bcc=14335&bt=37' 
r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml')
"""

cream_list = soup.find_all('div', class_='itemContainer')

url = []
for item in cream_list:
        for link in item.find_all('a', href=True): 
            url.append(link['href'])

In [3]:
eyecreams_url = []
for i in range (1, 14):
    url = f'https://www.yesstyle.com/en/women-eye-skin-care/list.html/bcc.14340_bpt.46#/bpt=46&sb=136&bt=37&bcc=14340&s=10&pn={i}&l=1' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    eyecreams_list = soup.find_all('div', class_='itemContainer')
    for item in eyecreams_list:
        for link in item.find_all('a', href=True): 
            eyecreams_url.append(link['href'])


In [4]:
print(len(eyecreams_url))

468


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [51]:
"""
test_url = 'https://www.yesstyle.com/es/cosrx-gel-limpiador-con-ph-bajo-good-morning-150ml/info.html/pid.1052684732'
r = requests.get(test_url) #para cada link en el listado, hacemos una requests
soup = BeautifulSoup(r.content, 'lxml')
name = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
for x in name:
    title = soup.find('h1').text
ingred = soup.find('div', class_="cropWrapper").text #ingredients
rating = soup.find('div', class_='satisfied').text
"""

In [6]:
eyecreams_url[467]

'https://www.yesstyle.com/en/pyunkang-yul-eye-cream-50ml-1ml-x-50pcs/info.html/pid.1067160762'

In [7]:
eyecreams = []

In [12]:
for link in eyecreams_url[0:468]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name_box = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
        for x in name_box:
            name = soup.find('h1').text
        ingredients = soup.find('div', class_="cropWrapper").text #ingredients
        rating = soup.find('div', class_='satisfied').text
    except:
        name= 'no info'
        ingredients = 'no info'
        rating = 'no info'
        
    eyecreams_info = {
    'name': name,
    'rating': rating,
    'ingredients': ingredients}
    
    eyecreams.append(eyecreams_info)

In [13]:
print(len(eyecreams))

468


In [81]:
eyecreams[100]

{'name': '\nKUMANO COSME - Pharmaact Deep Cleansing Oil\n',
 'rating': '97.3% Clientes satisfechos|317 reseñas',
 'ingredients': '\nMineral oil, PEG-8 glyceryl isostearate, water, glycerin, dimethicone, citric acid, jojoba oil, squalane, ethyl paraben, perfume\n\nMás\n\n'}

## 3. PASARLO A DATAFRAME

In [14]:
eyecreams_df = pd.DataFrame(eyecreams)
eyecreams_df

,name,rating,ingredients
0,\nMIZON - Snail Repair Eye Cream 25ml\n,"95.9% customers satisfied|3,108 reviews","\nSnail Secretion Filtrate, Glycerin, Butylene..."
1,\nMILKYDRESS - Wrinkle & Whitening Eye Cream 1...,"87.8% customers satisfied|2,730 reviews","\nWater, Butylene Glycol, Glycereth-26, Glycer..."
2,\nPURITO - Centella Green Level Eye Cream 30ml\n,95.6% customers satisfied|857 reviews,\nCentella Asiatica Extract (skin-conditioning...
3,\nBenton - Fermentation Eye Cream\n,"94.2% customers satisfied|1,803 reviews","\nGalactomyces Ferment Filtrate, Aqua (Water),..."
4,\nMIZON - Snail Repair Intensive Gold Eye Gel ...,94.6% customers satisfied|352 reviews,"\nWater, Glycerin, Calcium Chloride, Ceratonia..."
...,...,...,...
463,\nPETITFEE - Collagen & CoQ10 Hydrogel Eye Pat...,88.6% customers satisfied|106 reviews,"\nWater, glycerin, calcium chloride, butylene ..."
464,\niUNIK - Propolis Vitamin Eye Cream Set\n,94.4% customers satisfied|36 reviews,\n[Propolis Vitamin Eye Cream]:\nPropolis Extr...
465,\nSecret Key - Gold Premium First Eye Patch 60...,89.9% customers satisfied|694 reviews,"\nWater, Dipropylene Glycol, Butylene Glycol, ..."
466,\nMEDI-PEEL - Hyaluron Cica Peptide 9 Ampoule ...,100% customers satisfied|15 reviews,"\nWater, Glycerin, Carrageenan, Butylene Glyco..."


In [15]:
eyecreams = eyecreams_df.to_csv('yesstyle_data/eyecreams.csv')